In [1]:
import os
import json

import matplotlib.pyplot as plt

from bs4 import BeautifulSoup as BS
import requests
import pandas as pd
import numpy as np

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

from times import CreateTimeColumn

In [2]:
for file in os.listdir('JSONs_for_TP'):
    converted_data = dict(data=list())
    with open('JSONs_for_TP/' + file, 'r') as f:
        json_data = json.loads(f.read())
        keys = list(json_data.keys())    
        converted_data['keys_'] = keys
        print('keys for {}: {}'.format(file, keys))
        
        for num in json_data[keys[0]].keys():
            data = dict()
            
            for key in keys:
                if key == 'Lat' or key == 'lat':
                    val = json_data[key][num]
                    try:
                        float(val)
                    except:
                        print('file: {}, val for lat: {}'.format(file, val))
                        
                    data['lat'] = json_data[key][num]
                elif key == 'Long' or key == 'lng':
                    val = json_data[key][num]
                    try:
                        float(val)
                    except:
                        print('file: {}, val for lng: {}'.format(file, val))
                    data['lng'] = val
                else:
                    data[key] = json_data[key][num]
                
            converted_data['data'].append(data)
            
        f.close()
        
    with open('d3_jsons/' + file, 'w') as write_file:
        write_file.write(json.dumps(converted_data))
        write_file.close()

keys for earthquakes2.json: ['Year', 'Month', 'Day', 'lat', 'lng', 'Mag', 'Depth']
keys for earthquakes_original.json: ['UTC', 'Country', 'lat', 'lng', 'Depth_km', 'Mag', 'Secondary_Effects', 'PDE_Shaking_Deaths', 'PDE_Deaths', 'Utsu_Deaths', 'EM-DAT_Deahts', 'Other_Deaths', 'Max_Deaths', 'Year', 'Month', 'Day']
keys for eruptions.json: ['Year', 'Month', 'Day', 'lat', 'lng', 'ExplosivityIndexMax', 'End_Year', 'End_Month', 'End_Day', 'Continuing']
keys for tsunamis.json: ['Year', 'Month', 'Day', 'lat', 'lng', 'Magnitude', 'Focal_Depth', 'Max_Water_Height', 'Deaths', 'Total_Deaths']


In [3]:
time_column = CreateTimeColumn()
time_column.convert_the_times()

In [4]:
%%javascript
require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
    }
});

<IPython.core.display.Javascript object>

In [5]:
%%javascript
// console.log(require.s.contexts._.defined);
require(['d3'], function(d3){
    console.log('THIS IS d3: ', d3);
    console.log('d3 version: ', d3.version);
   
});


<IPython.core.display.Javascript object>

In [6]:
%%html
<style>
// add your CSS styling here
h1 {
    font-family: sans-serif;
    font-weight: 300;
    font-size: 16px;
}
</style>

In [7]:
%%html
<style>
svg {
    border: 1px solid #3f87a6;
},

h1 {
    font-family: sans-serif;
    font-weight: lighter;
}
</style>

In [12]:
%%javascript
require(['d3'], function(d3) {
    $('#chart1').remove();
    element.append(`
        <div class='container' style='width: 990px;'>
            <div id='chart1' class='row'>                
                <div class='col-md-8'>
                    <div class='row'>
                        <div class='col-md-6'>
                            <h1 id='map-title' style="font-family: sans-serif; font-weight: lighter">Earthquakes 1900-2011</h1>
                        </div>
                        <div class='col-md-4 col-md-offset-2'>
                            <div class="btn-group" role="group" aria-label="Time Lapse">
                                <button id='time-lapse-start' type="button" class="btn btn-secondary">Start</button>
                                <button id='time-lapse-stop' type="button" class="btn btn-secondary">Stop</button>
                                <button id='time-lapse-reset' type="button" class="btn btn-secondary">Reset</button>
                            </div>
                        </div>
                    </div>
                    <div class='row'>
                        <div id='map-container container' class='col-md-12'>
                            <div id='map'></div>
                        </div>                 
                    </div>
                </div>

                <div class='col-md-4'>
                    <div class='row'>
                        <div id='row-title' class='col-md-12'>
                        </div>
                    </div>
                    <div id='data-stats' class='row'>
                    </div>
                </div>
            </div>
            <div class='row checkbox-filters'>
                <div class="checkbox">
                    <label>
                      <input class="filter_button" id="eruptions_button" name="eruptions" type="checkbox">
                      Eruptions
                    </label>
                </div>
                <div class="checkbox">
                    <label>
                      <input class="filter_button" id="tsunamis_button" name="tsunamis" type="checkbox">
                      Tsunamis
                    </label>
                </div>
            </div>
        </div>
    `);
    
    var yearStats = function(year, month, day) {
        return `<h1>${year} / ${month} / ${day}</h1>`
    }
    
    var tsunamisStats = function(d) {
        return `<ul>
                    <li>Magnitude: ${d.Magnitude ? d.Magnitude : 'Unknown'}</li>
                    <li>Focal Depth: ${d.Focal_Depth ? d.Focal_Depth : 'Unknown'}</li>
                    <li>Max Water Height: ${d.Max_Water_Height ? d.Max_Water_Height : 'Unknown'}</li>
                    <li>Total Deaths: ${d.Total_Deaths ? d.Total_Deaths : 'Unknown'}</li>
                </ul>`;
    }
    
    var earthquakesStats = function(d) {
        return `<ul>
                    <li>Depth (km): ${d.Depth_km ? d.Depth_km : 'Unknown'}</li>
                    <li>Magnitude: ${d.Mag ? d.Mag : 'Unknown'}</li>
                    <li>Max Deaths: ${d.Max_Deaths ? d.Max_Deaths : 'Unknown'}</li>
                    <li>Country: ${d.Country ? d.Country : 'Unknown'}</li>
                </ul>`;
    }
    
    var eruptionsStats = function(d) {
        return `<ul>
                    <li>Explosivity Index Max: ${d.ExplosivityIndexMax ? d.ExplosivityIndexMax : 'Unknown'}</li>
                    <li>Continuing: ${d.Continuing ? d.Continuing : 'Unknown'}</li>
                </ul>`;
    }
    
    $(".filter_button").prop('checked', true);
//     <div class='container'>
//                 <div id='stats-container' class='col-md-4'>
//                     <div class='row'>
//                         <h2 id='stats-country'>Country: <span id='stats-country-dynamic'></span></h1>
//                         <div id='stats-earthquake-dynamic' class='row'><h2 id='stats-earthquake'>Earthquake: </h2></div>
//                         <div id='stats-volcanos-dynamic' class='row'><h2 id='stats-earthquake'>Volcano: </h2></div>
//                         <div id='stats-tsunamis-dynamic' class='row'><h2 id='stats-earthquake'>Tsunami: </h2></div>
//                     </div>                
//                 </div>
//             </div>
    
    function createAppendTooltip(d, name) {
        var $earthquakeStats = `<ul><li><strong>Time of event: ${d.day}/${d.month}/${d.year}</strong></li></ul>`;
        var idName = '#stats-' + name + '-dynamic';

        $( idName ).append( $earthquakeStats );
    }

    var width = 950,
        height = 400;
    
    function importData(filePath) {
        return new Promise((resolve, reject) => {
             d3.json(filePath, function(error, json) {
                if (json) { resolve(json); }
                reject(error);
            });
        });
    }
    
    var promisesArray = [
        importData('d3_jsons/times.json'), 
        importData('custom.geo.json'), 
        importData('d3_jsons/tsunamis.json'), 
        importData('d3_jsons/eruptions.json'), 
        importData('d3_jsons/earthquakes_original.json')
    ];
        
    // Loading in all of the files at once
    Promise.all(promisesArray).then(
        results => { 
            console.log('results', results); 
            
            var timeJson = results[0], 
                countriesJson = results[1],
                tsunamisJson = results[2],
                eruptionsJson = results[3],
                earthquakesJson = results[4];            
        
            var svg = d3.select('#map').append('svg')
                    .attr({ width: 650, height: height }),
                g = svg.append('g');
            
            // then, create the zoom behvavior
            var zoom = d3.behavior.zoom()
                // only scale up, e.g. between 1x and 50x
                .scaleExtent([1, 50])
                .on("zoom", function() {
                  // the "zoom" event populates d3.event with an object that has
                  // a "translate" property (a 2-element Array in the form [x, y])
                  // and a numeric "scale" property

                    var e = d3.event,
                        // now, constrain the x and y components of the translation by the
                        // dimensions of the viewport
                        tx = Math.min(0, Math.max(e.translate[0], width - width * e.scale)),
                        ty = Math.min(0, Math.max(e.translate[1], height - height * e.scale));

                    // then, update the zoom behavior's internal translation, so that
                    // it knows how to properly manipulate it on the next movement
                    zoom.translate([tx, ty]);
                    // and finally, update the <g> element's transform attribute with the
                    // correct translation and scale (in reverse order)
                    g.attr("transform", [
                        "translate(" + [tx, ty] + ")",
                        "scale(" + e.scale + ")"
                    ].join(" "));
                });

            svg.call(zoom).call(zoom.event);
            
            // Create a unit projection.
            var projection = d3.geo.mercator()
                .scale(1)
                .translate([0, 0]);

            // Create a path generator.
            var path = d3.geo.path()
                .projection(projection);

            // Compute the bounds of a feature of interest, then derive scale & translate.
            var b = path.bounds(countriesJson),
                s = 1.25 / Math.max((b[1][0] - b[0][0]) / width, ((b[1][1] - b[0][1]) / height) * 1.2),
                t = [(width - s * (b[1][0] + b[0][0])) * .68 / 2, (height - s * (b[1][1] + b[0][1])) / 2];        

            // Update the projection to use computed scale & translate.
            projection.scale(s).translate(t);
            
            
            // Draw each province as a path
            g.selectAll('path')
                .data(countriesJson.features)
                .enter().append('path')
                    .attr({
                        'd': path,
                        'id': function(d) {return d.properties.name;},
                        'class': function(d) { return ['continent', d.properties.continent]; }
                    })
                    .attr("data-legend", function(d) { return d.properties.continent })
                    .style({
                        'fill': '#666',
                        'stroke-opacity': .4,
                        'stroke': '#D3D3D3'
                    })
                    .on('mouseover', function(d) {
                        d3.select( this )
                            .transition()
                            .style({
                                'stroke-opacity': 1,
                                'stroke': '#f00',
                                'fill': 'pink'
                            });

                        $( '#stats-country-dynamic' ).text( d.properties.name ) ;
                    })
                    .on('mouseout', function(d) {
                        d3.select(this)
                            .transition()
                            .style({
                                'stroke-opacity': .4,
                                'stroke': '#D3D3D3',
                                'fill': '#666'
                            });

                        $('#stats-country-dynamic').text( ' ' );
                    });
            
            var dataConfig = {         
                tsunamis: { name: 'tsunamis', color: 'blue', circlePath: '.blue.dot', className: 'blue dot' }, 
                eruptions: { name: 'eruptions', color: 'red', circlePath: '.red.dot', className: 'red dot' },
                earthquakes: { name: 'earthquakes', color: 'green', circlePath: '.green.dot', className: 'green dot' }, 
            };            
            
            function createCircles(jsonData, dType) {
                let configData = dataConfig[dType];

                console.log('dType', dType, configData);                

                var showDots = true;
                if ( dType == 'tsunamis' || dType == 'eruptions') { 
                    $( configData.circlePath ).remove();
                    showDots = $( '#' + configData.name + '_button' ).prop( 'checked' ); 
                }

                if (showDots) {
                    g.selectAll(configData.circlePath)
                        .data(jsonData.data).enter()
                        .append('circle')
                        .attr({
                          cx: function(d) { 
                              if (d.lng && d.lat) { return projection([d.lng, d.lat])[0]; }
                              else { return projection(['0.0', '0.0'])[0]; }
                          },
                          cy: function(d) {
                              if (d.lng == 'NaN'|| d.lat == 'NaN') { console.log(d); }
                              if (d.lng && d.lat) { return projection([d.lng, d.lat])[1]; }
                              else { return projection(['0.0', '0.0'])[1]; }
                          },
                          r: function(d) { 
                              if (!d.lng && !d.lat && configData.color == 'green') { console.log('lat', d.lat, 'lng', d.lng); }
                              return (d.lng && d.lat) ? '2': '0'; 
                          },
                          'class': function(d) { 
                              var year;
                              
                              return configData.className + ' ' + d.t; }
                        })
                        .style({
                            'fill': configData.color,
                            'opacity': .6
                        })
                        .on('mouseover', function(d) {
                            if (configData.name == 'earthquakes') { createAppendTooltip(d, 'earthquakes'); }
                            else if (configData.name == 'tsunamis') { createAppendTooltip(d, 'tsunamis'); }
                            else if (configData.name == 'eruptions') { createAppendTooltip(d, 'eruptions'); }

                        })
                        .on('mouseout', function(d) {
                            if (configData.name == 'earthquakes') { $( '#stats-earthquakes-dynamic ul' ).remove(); }
                            else if (configData.name == 'tsunamis') { $( '#stats-tsunamis-dynamic ul' ).remove(); }
                            else if (configData.name == 'eruptions') { $( '#stats-eruptions-dynamic ul' ).remove(); }                            
                        });
                }
            }
            
            var circleData = [
                [earthquakesJson, 'earthquakes'],
                [eruptionsJson, 'eruptions'],
                [tsunamisJson, 'tsunamis'],
            ]
            
            circleData.forEach((args) => {
                createCircles(args[0], args[1]);
            });
            
            function hideShowCircles() {
                console.log(this.checked);
                if ( this.name == 'eruptions' ) { createCircles(eruptionsJson, 'eruptions'); }
                else if ( this.name == 'tsunamis' ) { createCircles(tsunamisJson, 'tsunamis'); }
            }

            d3.selectAll('.filter_button').on('click', hideShowCircles);
            
            var minTime = d3.min(timeJson), 
                maxTime = d3.max(timeJson),
                currentTime = minTime,
                yearInterval;
            
            console.log(minTime, maxTime);
            
            function startInterval() {
                console.log('starting interval');
                if (currentTime == 0) { $('circle').hide(); }
                
                yearInterval = setInterval(function() {
                    $('.' + currentTime).each(function() {
                        $( this ).show();
                        $('row-title').html(yearStats(this.__data__.year, this.__data__.month, this.__data__.day));
                    });
                    currentTime++
                    console.log('year index is being set!', currentTime);
                }, 50);
            }
            
            function stopYearInterval() { 
                console.log('stopping interval');
                var interval = yearInterval; 
                clearInterval(interval); 
            }       
            
            function resetTimes() {
                console.log('restarting interval');
                stopYearInterval();
                
                $('circle').show();
                $('row-title').empty();
                
                minTime = d3.min(timeJson);
                maxTime = d3.max(timeJson);
                currentTime = minTime;
            }
            
            $('#time-lapse-start').on('click', startInterval);
            $('#time-lapse-stop').on('click', stopYearInterval);
            $('#time-lapse-reset').on('click', resetTimes);
        
        },
        reason => { console.log('reason', reason ); }
    );
});

<IPython.core.display.Javascript object>